In [ ]:
from __future__ import absolute_import, division, print_function
from collections import defaultdict
import matplotlib.pyplot as plt
from chempy.chemistry import Reaction, Substance, ReactionSystem
from chempy.kinetics.ode import get_odesys
from chempy.util.graph import rsys2graph
import sympy
sympy.init_printing()
%matplotlib inline

In [ ]:
A, B, C = map(Substance, 'ABC')
One = sympy.S.One
reactions = r0, r1, r2 = [
    Reaction({'A'}, {'B'}, 4*One/100, name='R1: A cons.'),
    Reaction({'B', 'C'}, {'A', 'C'}, 10**(4*One), name='R2: A reform.'),
    Reaction({'B': 2}, {'B', 'C'}, 3*10**(7*One), name='R3: C form.')
]
rsys = ReactionSystem(reactions, (A, B, C))
rsys

In [ ]:
rsys2graph(rsys, 'robertson.png', save='.')
from IPython.display import Image; Image('robertson.png')

In [ ]:
odesys = get_odesys(rsys, include_params=True)[0]
odesys.exprs

In [ ]:
odesys.get_jac()

In [ ]:
c0 = defaultdict(float, {'A': 1})
odesys.integrate(1e10, c0, integrator='cvode')
odesys.plot_result()
plt.gca().set_xscale('log'); plt.gca().set_yscale('log'); _ = plt.legend(loc='best')